# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),             # Input shape matching MNIST image size
    tf.keras.layers.Flatten(),                         # Flatten 28x28 images to 1D vectors
    tf.keras.layers.Dense(64, activation='relu'),      # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer with 10 neurons (one per digit class)
])

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and measure training time
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")     # Print the training duration

# Evaluate the model on the test set
model.evaluate(x_test, y_test)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8580 - loss: 0.5002
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9560 - loss: 0.1562
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9659 - loss: 0.1145
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9738 - loss: 0.0879
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9788 - loss: 0.0723
TF Training time: 26.58 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9684 - loss: 0.1026


[0.09102629870176315, 0.9710000157356262]

## Convert TensorFlow model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp2zvkcv1_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140026886898512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140026886903504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140026886903696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140026886903888: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time  # ◄ Added to measure time

# Preprocess: Flatten MNIST images (28x28 → 784) and normalize to [0,1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # ◄ Flattens the image
])

# Load datasets
train_loader = DataLoader(
    datasets.MNIST(root='./data', train=True, transform=transform, download=True),
    batch_size=32,
    shuffle=True  # ◄ Shuffling for better training
)
test_loader = DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform, download=True),
    batch_size=1000
)

# Neural Network Architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)  # ◄ Input=784 (28*28), Output=256
        self.fc2 = nn.Linear(256, 10)   # ◄ Input=256, Output=10 (digits 0-9)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # ◄ ReLU activation after first layer
        return self.fc2(x)        # ◄ Output layer (no softmax: CrossEntropyLoss handles it)

model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # ◄ Added learning rate
loss_fn = nn.CrossEntropyLoss()

# Training loop
start = time.time()
for epoch in range(5):
    model.train()  # ◄ Set model to training mode
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(dim=1)  # ◄ Get predicted class
        correct += (pred == y).sum().item()

print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.49MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.82MB/s]


PyTorch Training time: 75.49 seconds
Test accuracy: 0.9781


## Convert PyTorch model to ONNX

In [ ]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 89.2 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values to [0, 1]
x_test = x_test / 255.0     # Same normalization for test set
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32             # Match batch size from first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # MNIST image size
    tf.keras.layers.Flatten(),                # Flatten to 784-dimensional vector
    tf.keras.layers.Dense(128, activation='relu'),    # Hidden layer with 128 neurons
    tf.keras.layers.Dense(10, activation='softmax')   # Output layer for 10 classes
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Added explicit learning rate
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.4435, Accuracy: 0.0625
Step 100, Loss: 0.3788, Accuracy: 0.7636
Step 200, Loss: 0.2483, Accuracy: 0.8277
Step 300, Loss: 0.1808, Accuracy: 0.8513
Step 400, Loss: 0.3741, Accuracy: 0.8666
Step 500, Loss: 0.4448, Accuracy: 0.8782
Step 600, Loss: 0.3491, Accuracy: 0.8850
Step 700, Loss: 0.3618, Accuracy: 0.8917
Step 800, Loss: 0.3253, Accuracy: 0.8967
Step 900, Loss: 0.2967, Accuracy: 0.9018
Step 1000, Loss: 0.2360, Accuracy: 0.9060
Step 1100, Loss: 0.2197, Accuracy: 0.9088
Step 1200, Loss: 0.1796, Accuracy: 0.9120
Step 1300, Loss: 0.4300, Accuracy: 0.9144
Step 1400, Loss: 0.0272, Accuracy: 0.9164
Step 1500, Loss: 0.0826, Accuracy: 0.9186
Step 1600, Loss: 0.1710, Accuracy: 0.9211
Step 1700, Loss: 0.1593, Accuracy: 0.9232
Step 1800, Loss: 0.2068, Accuracy: 0.9249
Training Accuracy for epoch 1: 0.9261

Epoch 2/5
Step 0, Loss: 0.1660, Accuracy: 0.9375
Step 100, Loss: 0.1919, Accuracy: 0.9616
Step 200, Loss: 0.0921, Accuracy: 0.9596
Step 300, Loss: 0.2784, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values to [0, 1]
x_test = x_test / 255.0     # Same normalization for test set
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model with improved architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # MNIST image size (28x28)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),    # Increased neurons for better learning
    tf.keras.layers.Dropout(0.2),                    # Regularization to prevent overfitting
    tf.keras.layers.Dense(128, activation='relu'),   # Additional hidden layer
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Explicit learning rate
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # Compile the function into a graph for faster execution
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 200 == 0:  # Reduced frequency of printing for cleaner output
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.3228, Accuracy: 0.1250
Step 200, Loss: 0.2773, Accuracy: 0.8197
Step 400, Loss: 0.2494, Accuracy: 0.8632
Step 600, Loss: 0.1857, Accuracy: 0.8862
Step 800, Loss: 0.1535, Accuracy: 0.9004
Step 1000, Loss: 0.0434, Accuracy: 0.9091
Step 1200, Loss: 0.0599, Accuracy: 0.9145
Step 1400, Loss: 0.2080, Accuracy: 0.9195
Step 1600, Loss: 0.0333, Accuracy: 0.9239
Step 1800, Loss: 0.1874, Accuracy: 0.9277
Training Accuracy for epoch 1: 0.9290

Epoch 2/5
Step 0, Loss: 0.2064, Accuracy: 0.9688
Step 200, Loss: 0.1068, Accuracy: 0.9661
Step 400, Loss: 0.2689, Accuracy: 0.9651
Step 600, Loss: 0.0189, Accuracy: 0.9639
Step 800, Loss: 0.0067, Accuracy: 0.9648
Step 1000, Loss: 0.1468, Accuracy: 0.9651
Step 1200, Loss: 0.0602, Accuracy: 0.9660
Step 1400, Loss: 0.1017, Accuracy: 0.9658
Step 1600, Loss: 0.0345, Accuracy: 0.9658
Step 1800, Loss: 0.0314, Accuracy: 0.9663
Training Accuracy for epoch 2: 0.9666

Epoch 3/5
Step 0, Loss: 0.0351, Accuracy: 1.0000
Step 200, Loss: 0.0566, Ac